In [ ]:
# Import packages.
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose, Resize

In [ ]:
# Download data.
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=Compose([ToTensor(), Resize((32, 32))]))
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=Compose([ToTensor(), Resize((32, 32))]))

In [ ]:
# Load data.
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
# Define model.
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.r1 = nn.ReLU()
        self.p1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.c2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.r2 = nn.ReLU()
        self.p2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.fl = nn.Flatten()
        
        self.l1 = nn.Linear(in_features=400, out_features=120)
        self.l2 = nn.Linear(in_features=120, out_features=84)
        self.l3 = nn.Linear(in_features=84, out_features=10)


    def forward(self, x):
        x = self.c1(x)
        x = self.r1(x)
        x = self.p1(x)
        
        x = self.c2(x)
        x = self.r2(x)
        x = self.p2(x)
        
        x = self.fl(x)
        
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        
        x = F.softmax(x, dim=1)
            
        return x

In [ ]:
# Set device.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [ ]:
# Instantiate model to device.
model = LeNet5().to(device)

In [ ]:
# Hyperparameters.
learning_rate = 1e-3
batch_size = 32
epochs = 25 

In [ ]:
# Loss function and optimizer.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train loop.
# Copied from: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Test loop.
# Copied from: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# Train.
# Copied from: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")